In [359]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_2022_02_03_dark',
           'manipulated_2023_08_10_100ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_07_10ms',
           'manipulated_2023_08_07_5ms',
] 
color_map = colormap.datad["tab10"]['listed']


experiments = {exp:Experiment(loadir,exp) for exp in exp_name}
[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot')  for exp in experiments.values()]
zero_velocity = [exp.zero_velocity_movies('CM_dot_projected') for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()]
# [exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments]




[None, None, None, None, None, None, None]

Calculate first peak/ min/ max

In [146]:
fig = go.Figure()
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','peaks',t0 = 5, t1 = 50) for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','peaks',t0 = 40, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max',t0 = 40, t1 = 100) for exp in experiments.values()]

[exp.get_peaks_movies('CM_dot_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_projected','zero_v') for exp in experiments.values()]



[None, None, None, None, None, None, None]

Plot histograms of peaks, plot movies time seires data with the max/min/peak data point on them

In [347]:
# Histograms
point_name = 'pitch_y_frame_dot_dot_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

point_name = 'pitch_y_frame_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_min','pitch_y_frame',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_max','pitch_y_frame_dot_dot',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_projected_min','CM_dot_dot_projected',fig) 
fig.show()

# 


3D trajectory plot 

In [358]:
exp = experiments['manipulated_2022_02_03_dark']
mov = ['mov71']
exp.plot_3d_traj_movies('CM_dot_projected',mov = mov)


In [343]:

import cmath
input_num = complex(1, 2) # stored as 1+2j


def rotate_vectors_to_reference(vectors, reference_vector):
    # Normalize the reference vector
    reference_vector = reference_vector / np.linalg.norm(reference_vector)
    
    # Calculate the rotation matrix
    rotation_matrix = np.eye(2) + np.outer(reference_vector, reference_vector) - np.outer(reference_vector, reference_vector)
    
    # Rotate the vectors
    rotated_vectors = vectors @ rotation_matrix.T
    
    return rotated_vectors

# Example vectors
vectors = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])

t0 = 20
t1 = 40
fig, ax = plt.subplots()
ang_exp = []

for exp_name in list(experiments.keys()): #experiments['manipulated_2023_08_06_40ms']
    ang= []

    exp = experiments[exp_name]
    [exp.project_prop_movies('X_x_body',wing_body = 'vectors',header_name = 'X_x_body')  for exp in experiments.values()]

    for mov_name in exp.mov_names:
        vec= [None]*2
        mov = exp.get_mov(mov_name)
        mov.get_prop('X_x_body_projected','vectors')
        try:
            for idx,(t,color) in enumerate(zip([(-10,10),(100,110)],['red','blue','black'])):
                
                data = mov.get_prop('X_z_body','vectors', three_col= 3)[1:,:]
                vector_to_project = mov.get_prop('X_z_body','vectors',three_col = 3)
                ref_axes = vector_to_project[mov.ref_frame,:]
                x_axis_on_xy = (ref_axes - ref_axes * [0,0,1])/np.linalg.norm(ref_axes-ref_axes * [0,0,1] )[np.newaxis].T # project the new axis to XY plane
                data_axis_on_xy = (data[:,0:2])/np.linalg.norm(data[:,0:2],axis = 1)[np.newaxis].T # project the new axis to XY plane

                # Example reference vector (one of the vectors in the list)
                reference_vector = vectors[0]
                idx_t0,idx_t1 = mov.t0_t1_idx(t[0],t[1])

                # Rotate the vectors to be in the frame of axes defined by the reference vector
                rotated_vectors = rotate_vectors_to_reference(data_axis_on_xy[:,0:2], x_axis_on_xy[0:2])[idx_t0:idx_t1,:]
                vec[idx] = np.nanmean(rotated_vectors,axis = 0)

                # Convert 3D vectors to polar coordinates
                angles = np.arctan2(rotated_vectors[:, 1], rotated_vectors[:, 0])
                magnitudes = np.linalg.norm(rotated_vectors, axis=1)

                quiveropts = dict( headlength=5, scale=20, 
                    linewidth=.5, units='xy', width=.0005, headwidth=5,color = color) # common options
                # Create a polar plot with arrows
                # 
                skip = 20
                # quiver = ax.quiver(rotated_vectors[::skip,0]*0, rotated_vectors[::skip,1]*0, rotated_vectors[::skip,0], rotated_vectors[::skip,1], headaxislength=4.5,alpha = 0.5, **quiveropts)
                # plt.title(mov.name)

        except:
            continue
        ang.append(np.arccos((vec[0] @ vec[1]/(np.linalg.norm(vec[0])*np.linalg.norm(vec[1]))))*180/np.pi)
    ang_exp.append(ang)
# # Create a polar plot with arrows
# fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
# quiver = ax.quiver(angles*0, magnitudes*0, angles, magnitudes, units='xy', scale=1, color='blue', width=0.01)


In [344]:
plt.hist(ang_exp[0:2],bins = 50,label = list(experiments.keys())[0:2],stacked= True, density= True)
plt.legend()

In [41]:
data_axis_on_xy = (data[:,0:2])/np.linalg.norm(data[:,0:2],axis = 1)[np.newaxis].T # project the new axis to XY plane
x_axis_on_xy

array([-0.597548  , -0.80183315,  0.        ])

In [41]:
exp = experiments['manipulated_2023_08_10_100ms']

point_name = 'CM_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_projected_min','CM_dot_projected',fig) 
fig.show()

In [39]:
exp = experiments['manipulated_2023_08_10_100ms']

point_name = 'pitch_body_peaks'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


fig = go.Figure()
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_body_peaks','pitch_body',fig) 
fig.show()

In [22]:

fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_projected_min','CM_dot_dot_projected',fig) 
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time',yaxis = 'y2')
exp.add_point_to_plot_movies('CM_dot_dot_projected_min','pitch_y_frame_dot_dot',fig,yaxis = 'y2') 

# yax_dict[f"yaxis2"] = (dict(title=prop,titlefont=dict(color=color),tickfont=dict(color=color),side = side[i],position=position[i]))




fig.update_layout(
    xaxis=dict(domain=[0, 0.95]),
    yaxis=dict(title='CM_dot_dot_projected', titlefont=dict(color='red'), tickfont=dict(color='red'), side='left', position=0),
    yaxis2=dict(title='pitch_body', titlefont=dict(color='green'), tickfont=dict(color='green'), side='right', position=0.95),
)

fig.show()

Plot all experiments - time series

In [6]:
fig = go.Figure()
prop = 'pitch_y_frame_dot_dot'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

Calculate and plot mean property (time series,  mean per frame of all movies)

In [25]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'r'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','q']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

Try PCA

In [19]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_projected','CM_dot_projected','pitch_y_frame_dot_dot']

exp_pd = []
for exp in list(experiments.values())[0:3]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(np.nanmean(mov.data['body'][15:60,:],axis = 0))

    featpd_exp = pd.DataFrame(data = mov_feat,columns = mov.header['body'])
    featpd_exp['exp_name'] = exp.experiment_name
    featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = 2
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



[0 0 0 0 0 0 0 2 2 2 0 0 2 1 0 0 1 0 2 2 2 2 2 0 0 2 2 2 0 0 0 2 0 0 2 2 0
 0 2]


array([[ 2.58930919e-03, -3.21593360e-07, -9.99996648e-01],
       [-9.99996645e-01, -7.69076209e-05, -2.58930916e-03]])

In [18]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name]['mov'])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Experiment.py:112: RuntimeWarning:

Mean of empty slice



Plot and save figures

In [ ]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,add_horizontal_line = False):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,'body',color_map[0],fig,mov = False,case = 'plot_mov')
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp.plot_3d_traj_movies('CM_dot_dot_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp,yprop = 'CM_dot_dot_projected',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'pitch_y_frame',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'pitch_y_frame_dot',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'CM_dot_projected',wing_body = 'body',add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')

